In [1]:
import pandas as pd
import numpy as np
from solar.backtesting import Backtesting
import pycaret
from pycaret.classification import *
from tqdm import tqdm_notebook as tqdm

In [2]:
raw_df = pd.read_csv('TIMESERIES_all_monthly_enriched_no_dairyntel.csv').ffill()
raw_df['date'] = pd.to_datetime(raw_df.apply(lambda x: str(x['date'])[:10],axis=1))
target_var = "ZuivelNL - Dutch SMP food quotation"
# target_var = "ZuivelNL - Dutch butter quotation"

In [3]:
def remove_invalid_features(df, min_num_valid=100):
    v_check = df.ffill().isna().sum() < len(df) - min_num_valid 
    v_feats = v_check[v_check].index.tolist()
    return [f for f in v_feats if len(df[f].unique()) > 1]

def prepare_data(time_horizon, target_var):
    input_df = raw_df.copy() 
    input_df['Target'] = -(input_df[target_var].diff(-time_horizon)>=0)
    valid_feats = remove_invalid_features(input_df, 99+time_horizon)
    data = input_df[valid_feats].dropna()
    data['Target'] = data['Target'].astype('int')
    return data

In [4]:
def run_pycaret(time_horizon,params):
    data = prepare_data(time_horizon, target_var)
    exp_clf = setup(data, **params)
#     best = compare_models(blacklist = ['catboost','gbc','ada','qda','ridge','svm'])
    best = create_model('lr')
    print(best)
    best_model = tune_model(best,n_iter=100)
    feature_list = exp_clf[0].columns
    bt_clf = Backtesting(data, best_model, time_horizon, feature_list, max_iterations=36, target_var=target_var, task = 'classification')
    return best_model, feature_list, bt_clf.run(train_length=60,test_length=1)[1]


def optimize(time_horizon, iterations = 10):
    best_acc = 0
    best_model = {}
    best_features = {}
    for i in tqdm(range(iterations)):
        params = {
        'target': 'Target',
        'train_size':0.9,
        'normalize' : True,
        'silent':True,
        'feature_selection':True,
        'feature_selection_threshold':0.001,
        'verbose':False
        }
        model, features, acc_df = run_pycaret(time_horizon, params)
        acc = acc_df.backtesting_accuracy[0]
        print(f'time_horizon:{time_horizon} bt_acc:{acc} num_features:{len(features)}')
        if acc > best_acc:
            best_acc = acc
            best_features = features
            best_model = model
    return best_acc, best_features, best_model

In [5]:
best_models = {}
best_features = {}
for time_horizon in [1,2,3,4,5,6,7,8,9]:
    acc, features, model = optimize(time_horizon)
    best_models[time_horizon] = model
    best_features[time_horizon] = features

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8000,0.8000,0.8000,0.8000,0.8000,0.6000,0.6000
1,0.8000,1.0000,1.0000,0.7143,0.8333,0.6000,0.6547
2,0.7000,0.8800,0.8000,0.6667,0.7273,0.4000,0.4082
3,0.9000,0.8800,0.8000,1.0000,0.8889,0.8000,0.8165
4,0.9000,0.9200,1.0000,0.8333,0.9091,0.8000,0.8165
5,0.8000,0.7917,0.6667,1.0000,0.8000,0.6154,0.6667
6,0.9000,0.8750,0.8333,1.0000,0.9091,0.8000,0.8165
7,0.9000,0.8750,1.0000,0.8571,0.9231,0.7826,0.8018
8,0.7778,0.8000,0.8000,0.8000,0.8000,0.5500,0.5500
9,0.7778,0.7500,1.0000,0.7143,0.8333,0.5263,0.5976


time_horizon:9 bt_acc:0.6111111111111112 num_features:6



In [6]:
acc_df = pd.DataFrame()
for time_horizon in [1,2,3,4,5,6,7,8,9]:
    data = prepare_data(time_horizon, target_var)
    bt_clf = Backtesting(data, best_models[time_horizon], time_horizon, best_features[time_horizon], max_iterations=36, target_var=target_var, task = 'classification')
    acc_df = acc_df.append(bt_clf.run(train_length=60,test_length=1)[1])
acc_df

,time_horizon,task,backtesting_accuracy,sliding,backtesting_iterations,backtesting_training_length,backtesting_test_length,backtesting_end_feature_date,backtesting_start_feature_date,backtesting_end_target_date,backtesting_start_target_date
0,1,classification,0.750000,False,36,60,1,2020-05-01,2017-06-01,2020-06-01,2017-07-01
0,2,classification,0.833333,False,36,60,1,2020-04-01,2017-05-01,2020-06-01,2017-07-01
0,3,classification,0.888889,False,36,60,1,2020-03-01,2017-04-01,2020-06-01,2017-07-01
0,4,classification,0.666667,False,36,60,1,2020-02-01,2017-03-01,2020-06-01,2017-07-01
0,5,classification,0.805556,False,36,60,1,2020-01-01,2017-02-01,2020-06-01,2017-07-01
0,6,classification,0.833333,False,36,60,1,2019-12-01,2017-01-01,2020-06-01,2017-07-01
0,7,classification,0.888889,False,36,60,1,2019-11-01,2016-12-01,2020-06-01,2017-07-01
0,8,classification,0.888889,False,36,60,1,2019-10-01,2016-11-01,2020-06-01,2017-07-01
0,9,classification,0.888889,False,36,60,1,2019-09-01,2016-10-01,2020-06-01,2017-07-01


In [10]:
{time_horizon:list(features) for time_horizon,features in best_features.items()}

{1: ['US SMP supply_1m_lag',
  'USDA US butter price_diff1_yoy',
  'ZuivelNL - Dutch SMP feed quotation_diff1',
  'US milk supply_4m_lag',
  'ARG milk supply_diff1_yoy',
  'NZ stocks-to-use_diff1',
  'Sossna Rohmilch avg_9m_lag'],
 2: ['Southern Oscillation Index_2m_lag',
  'Sossna flussig avg_diff2_yoy',
  'ZuivelNL - US butter price',
  'Sossna flussig S_diff2_yoy',
  'USDA US cheddar price_8m_lag',
  'SMP_basket_valorization'],
 3: ['ZuivelNL - Dutch SMP feed quotation_ra12m',
  'Sossna flussig avg_diff1',
  'ARG milk supply_diff2_yoy',
  'USD/DZD_diff1_yoy',
  'ARG milk supply_diff1_yoy',
  'cheese_vs_smp_bucket_valorization_rv'],
 4: ['ZuivelNL - Dutch SMP food quotation_10m_lag',
  'ZuivelNL - NZ butter price_9m_lag',
  'AUS milk supply_7m_lag',
  'ZuivelNL - NZ butter price_diff1_yoy',
  'Sossna MM Konz avg_12m_lag',
  'ZuivelNL - NZ cheddar price_rv',
  'ZuivelNL - US cheddar block price_1m_lag',
  'ZuivelNL - world WMP price_diff2_yoy',
  'USD/DZD_diff1_yoy',
  'ZuivelNL - wor

In [8]:
best_models

{1: LogisticRegression(C=0.053, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                    random_state=7397, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 2: LogisticRegression(C=8.113, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                    random_state=7176, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 3: LogisticRegression(C=0.35000000000000003, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                    random_state=5481, solver='lbfgs', tol=0.0001, verbose=0,
  